<a href="https://colab.research.google.com/github/2004prajapatirohit/Efficient-Relay-Selection-and-Transmission-Scheduling-in-NTN-Assisted-Vehicular-Networks/blob/main/Efficient_Relay_Selection_and_Transmission_Scheduling_in_NTN_Assisted_Vehicular_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Define the Scheduling Environment

In [ ]:

import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import gym
import pandas as pd
import math
import csv

len_flows = 500
max_slots = 12500
Qa = 0.5

class SchedulingEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Discrete(len_flows)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(len_flows,), dtype=np.float32)
        self.positions = self._generate_positions()
        self.satellite_flows = []
        self.flows = self._generate_flows()
        self.los_data = self._generate_los_data()
        self.time_slots = 0
        self.state = self._get_initial_state()
        self.scheduled_flows = []
        self.link_data = []
        self.schedule = {i: [] for i in range(len_flows)}
        self.throughput_cache = {}

    def _generate_positions(self):
        car_length = 4.5
        car_width = 2
        lanes = [0, 3.75, 7.5]
        rate = 0.03

        positions = []
        for lane in lanes:
            current_x = 0
            while current_x < 3500 and len(positions) < 60:
                gap = np.random.exponential(scale=1.0 / rate)
                current_x += gap + car_length
                if current_x < 3000:
                    positions.append((current_x - car_length / 2, lane))

        positions = np.array(positions)
        np.random.shuffle(positions)
        return positions[:100]

    def _UAV_positions(self):
        return [(500, 3.75), (1500, 3.75), (2500, 3.75)]

    def _generate_flows(self):
        flows = set()
        while len(flows) < len_flows:
            tx = random.choice(range(60))
            rx = random.choice([i for i in range(60) if i != tx])
            pos1 = self.positions[tx]
            pos2 = self.positions[rx]

            if self._line_of_sight(pos1, pos2):
                flows.add((tx, rx))
            else:
                uav_supported = False
                for uav in self._UAV_positions():
                    if self._line_of_sight_uav(pos1, pos2, uav):
                        uav_supported = True
                        flows.add((tx, rx))
                        break

                if not uav_supported:
                    flows.add((tx, rx))
                    self.satellite_flows.append((tx, rx))

        return list(flows)

    def _get_initial_state(self):
        return np.zeros(len_flows)

    def _line_of_sight_uav(self, pos1, pos2, uav):
        return abs(pos1[0] - uav[0]) <= 500 and abs(pos2[0] - uav[0]) <= 500

    def _line_of_sight(self, pos1, pos2):
        for pos in self.positions:
            if not (np.all(pos == pos1) or np.all(pos == pos2)):
                if self._is_intersecting(pos1, pos2, pos):
                    return False
        return True

    def _is_intersecting(self, pos1, pos2, pos):
        car_length = 4.5
        car_width = 2
        car_bbox = [
            (pos[0] - car_length / 2, pos[1] - car_width / 2),
            (pos[0] + car_length / 2, pos[1] - car_width / 2),
            (pos[0] + car_length / 2, pos[1] + car_width / 2),
            (pos[0] - car_length / 2, pos[1] + car_width / 2),
        ]
        for i in range(4):
            if self._do_intersect(pos1, pos2, car_bbox[i], car_bbox[(i + 1) % 4]):
                return True
        return False

    def _do_intersect(self, p1, p2, q1, q2):
        def orientation(p, q, r):
            val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
            return 0 if val == 0 else (1 if val > 0 else 2)

        def on_segment(p, q, r):
            return (min(p[0], r[0]) <= q[0] <= max(p[0], r[0]) and
                    min(p[1], r[1]) <= q[1] <= max(p[1], r[1]))

        o1 = orientation(p1, p2, q1)
        o2 = orientation(p1, p2, q2)
        o3 = orientation(q1, q2, p1)
        o4 = orientation(q1, q2, p2)

        if o1 != o2 and o3 != o4: return True
        if o1 == 0 and on_segment(p1, q1, p2): return True
        if o2 == 0 and on_segment(p1, q2, p2): return True
        if o3 == 0 and on_segment(q1, p1, q2): return True
        if o4 == 0 and on_segment(q1, p2, q2): return True

        return False

    def _generate_los_data(self):
        los_data = []
        for tx, rx in self.flows:
            pos1 = self.positions[tx]
            pos2 = self.positions[rx]
            los = 1 if self._line_of_sight(pos1, pos2) else 0
            los_data.append(los)
        return np.array(los_data)

    def _V2V_Time(self, flow):
        tx, rx = flow
        G0 = 10 ** (20 / 10)
        P_t = 10**((40 - 30) / 10)
        alpha_v = 2.5
        wavelength = (3 * 10**8) / (30 * 10**9)
        k_v = (wavelength / (4 * math.pi)) ** alpha_v
        N0 = -134
        W = 2000
        beta = 10**(-110 / 10)
        eta = 0.8
        Q_a = Qa
        M = max_slots

        def distance(p1, p2):
            return np.linalg.norm(np.array(p1) - np.array(p2))

        d = distance(self.positions[tx], self.positions[rx])
        P_r = k_v * P_t * G0 * d ** -alpha_v
        SNR = P_r / (10 ** ((N0 + 10 * np.log10(W) - 30) / 10))
        rate = eta * W * np.log2(1 + SNR) * 1e-3
        return math.ceil(Q_a * M / rate)

    def _V2U_Time(self, tx, uav):
        G0_dBi = 20  # Placeholder value for the beamforming gain
        G0 = 10 ** (G0_dBi / 10)
        theta_3dB = 30  # Placeholder value for the 3dB angle
        P_t_dBm = 40  # Transmit power
        P_t = 10**((P_t_dBm-30)/10)
        alpha_u = 2  # Path loss exponent
        carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
        wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
        k_u = (wavelength / (4 * math.pi)) ** alpha_u  # Constant factor for power calculations
        N0 = -134  # Noise power spectral density
        W = 2000  # Bandwidth (MHz)
        eta = 0.8  # Spectral efficiency
        Q_a = Qa  # Placeholder for the amount of data to be transmitted
        M = max_slots  # Placeholder for the number of transmissions
        T = 0.1  # Duration of one time slot

        def distance(tx, uav):
            return np.cbrt((tx[0] - uav[0]) ** 2 + (tx[1] - uav[1]) ** 2  + 100 ** 2)  # Access tuple elements for subtraction

        g = 1  # Placeholder for channel gain
        d_s_r = distance(tx, uav)

        P_r_s_r = k_u * P_t * G0 * g * d_s_r ** -alpha_u

        SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
        rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps
        time_slots = math.ceil(Q_a * M / (rate))

        return time_slots

    def satellite_time(self,tx_pos, rx_pos):
        G_es_dBi = 48
        P_es_dBm = 44
        G_es = 10 ** (G_es_dBi / 10)
        P_es = 10 ** ((P_es_dBm - 30) / 10)

        G_sat_tx_dBi = 38.5
        G_ship_rx_dBi = 2
        P_sat_dBm = 40
        G_sat_tx = 10 ** (G_sat_tx_dBi / 10)
        G_ship_rx = 10 ** (G_ship_rx_dBi / 10)
        P_sat = 10 ** ((P_sat_dBm - 30) / 10)

        alpha_sat = 2
        carrier_frequency = 30e9
        wavelength = 3e8 / carrier_frequency
        k_sat = (wavelength / (4 * math.pi)) ** alpha_sat

        N0_dBm_Hz = -174
        BW = 2000e6
        N0_total_dBm = N0_dBm_Hz + 10 * np.log10(BW)
        N0 = 10 ** ((N0_total_dBm - 30) / 10)

        eta = 0.8
        h_sat = 1200e3
        d = math.sqrt((tx_pos[0] - rx_pos[0]) ** 2 + (tx_pos[1] - rx_pos[1]) ** 2)
        R = math.sqrt(h_sat ** 2 +(tx_pos[0] - 1500) ** 2 + (tx_pos[1]) ** 2)

        g = 1
        P_r_uplink = k_sat * P_es * G_es * G_sat_tx * g * R ** -alpha_sat
        SNR_uplink = P_r_uplink / N0
        rate_uplink = eta * BW * np.log2(1 + SNR_uplink) * 1e-6
        R1 = math.sqrt(h_sat ** 2 +(rx_pos[0] - 1500) ** 2 + (rx_pos[1]) ** 2)
        P_r_downlink = k_sat * P_sat * G_sat_tx * G_ship_rx * g * R1 ** -alpha_sat
        SNR_downlink = P_r_downlink / N0
        rate_downlink = eta * BW * np.log2(1 + SNR_downlink) * 1e-6

        rate = min(rate_uplink, rate_downlink)
        return math.ceil(Qa * max_slots / rate)

    def check_flow(self, picked_flows, flow_i):
        arr = np.ones(len(picked_flows), dtype=int)
        tx_i, rx_i = flow_i
        for j, flow_j in enumerate(picked_flows):
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                arr[j] = 0
        return arr

    def step(self, action):
        self.state[action] = 1
        scheduled_flow = self.flows[action]
        time_slots = self._V2V_Time(scheduled_flow)
        self.time_slots += time_slots
        self.schedule[action].append(self.time_slots)
        done = np.all(self.state == 1)
        return self.state, done, {}

    def reset(self):
        self.state = self._get_initial_state()
        self.time_slots = 0
        self.scheduled_flows = []
        self.link_data = []
        self.schedule = {i: [] for i in range(len_flows)}
        return self.state

# === Instantiate environment and run ===
env = SchedulingEnv()
state = env.reset()
positions = env.positions
flows = env.flows
los_data = env.los_data

print("LOS data:", los_data)

Time_Slots = np.array([env._V2V_Time(flow) for flow in flows])
print(f"Time_Slots: {Time_Slots}")

flows_df = pd.DataFrame({'Flow': [f"{tx}->{rx}" for tx, rx in flows]})
throughput_df = pd.DataFrame({'Time Slots': Time_Slots})
data_df = pd.concat([flows_df, throughput_df], axis=1)
data_df.to_csv('/content/data.csv', index=False)

los_data_df = pd.DataFrame({'LOS': los_data})
los_data_df.to_csv('/content/los_data.csv', index=False)

positions_list = [tuple(pos) for pos in positions]
positions_df = pd.DataFrame({'Positions': positions_list})
positions_df.to_csv('/content/positions_data.csv', index=False)


LOS data: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 

Generate Data For Non-UAV and Non-Satellite

In [ ]:

import pandas as pd
import ast
import math
import numpy as np

# Initialize all variables to empty lists
flows = []
time_slots = []
los_data = []
time_slots_old = []
positions = []
positions_old = []
flows_old = []

# Load the CSV files
data_df = pd.read_csv('/content/data.csv')
los_data_df = pd.read_csv('/content/los_data.csv')
positions_data_df = pd.read_csv('/content/positions_data.csv')

# Extract tx and rx from the 'Flow' column
flows = [(int(flow.split('->')[0]), int(flow.split('->')[1])) for flow in data_df['Flow']]

# Extract time slots from the 'Time Slots' column
time_slots_old = data_df['Time Slots'].to_dict()

# Extract LOS data
los_data = los_data_df['LOS'].to_dict()

# Remove time slots greater than 10000 and corresponding data in LOS and flows
filtered_time_slots = {k: v for k, v in time_slots_old.items() if 5 <= v <= 500}

# Keep only the flows and LOS data corresponding to the filtered time slots
filtered_flows = [flows[i] for i in filtered_time_slots.keys()]
filtered_los_data = {k: los_data[k] for k in filtered_time_slots.keys()}

# Update the original variables with the filtered data
time_slots = list(filtered_time_slots.values())
flows = filtered_flows
flows_old = flows.copy()  # Use copy() to avoid changing flows_old if flows is modified later
los_data = filtered_los_data

# Convert the 'Positions' column back to a list of tuples
# Use eval to handle NumPy objects
positions_list = [eval(pos) for pos in positions_data_df['Positions']]
positions_old = [list(pos) for pos in positions_list]
positions = [list(pos) for pos in positions_list]

def _V2V_Time(flow):
    # Calculate throughput using NumPy vectorization
    tx, rx = flow
    G0_dBi = 20  # Placeholder value for the beamforming gain
    G0 = 10 ** (G0_dBi / 10)
    theta_3dB = 30  # Placeholder value for the 3dB angle
    P_t_dBm = 40  # Transmit power
    P_t = 10**((P_t_dBm-30)/10)
    alpha_v = 2.5  # Path loss exponent
    carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
    wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
    k_v = (wavelength / (4 * math.pi)) ** alpha_v  # Constant factor for power calculations
    N0 = -134  # Noise power spectral density
    W = 2000  # Bandwidth (MHz)
    eta = 0.8  # Spectral efficiency
    Q_a = 0.1  # Placeholder for the amount of data to be transmitted
    M = max_slots  # Placeholder for the number of transmissions
    T = 0.1  # Duration of one time slot

    def distance(pos1, pos2):
        return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

    g = 1  # Placeholder for channel gain
    d_s_r = distance(positions[tx], positions[rx])

    P_r_s_r = k_v * P_t * G0 * g * d_s_r ** -alpha_v
    SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps

    time_slots = math.ceil(Q_a * M / (rate))
    print(f"Time Slots: {time_slots}")

    return time_slots


def _V2U_Time(tx, uav):
    G0_dBi = 20  # Placeholder value for the beamforming gain
    G0 = 10 ** (G0_dBi / 10)
    theta_3dB = 30  # Placeholder value for the 3dB angle
    P_t_dBm = 40  # Transmit power
    P_t = 10**((P_t_dBm-30)/10)
    alpha_u = 2  # Path loss exponent
    carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
    wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
    k_u = (wavelength / (4 * math.pi)) ** alpha_u  # Constant factor for power calculations
    N0 = -134  # Noise power spectral density
    W = 2000  # Bandwidth (MHz)
    eta = 0.8  # Spectral efficiency
    Q_a = 0.1  # Placeholder for the amount of data to be transmitted
    M = max_slots  # Placeholder for the number of transmissions
    T = 0.1  # Duration of one time slot

    def distance(tx, uav):
        return np.cbrt((tx[0] - uav[0]) ** 2 + (tx[1] - uav[1]) ** 2 + 100 ** 2)  # Access tuple elements for subtraction

    g = 1  # Placeholder for channel gain
    d_s_r = distance(tx, uav)

    P_r_s_r = k_u * P_t * G0 * g * d_s_r ** -alpha_u

    SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps
    Q_a = 1  # Gbps
    time_slots = math.ceil(Q_a * M / (rate))

    return time_slots

# Initialize groups outside the loop
group1 = []
group0 = []
group = []
rejects = []

# Process the flows and LOS data
for flow_index, (tx, rx) in enumerate(flows_old):
    if los_data.get(flow_index) == 0:
        time_uav = 0
        time_v2v = 0

        # Find positions for tx and rx
        tx_pos = positions[tx] if tx < len(positions_old) else None
        rx_pos = positions[rx] if rx < len(positions_old) else None

        # Check if tx_pos and rx_pos are valid
        if tx_pos and rx_pos:
            # Initialize tx_list and rx_list as sets to avoid duplicates
            tx_list = set()
            rx_list = set()

        # Check for UAV hop
        tx_x = tx_pos[0]
        rx_x = rx_pos[0]

        # Determine the intermediate UAV position
        if 0 <= tx_x <= 1000 and 0 <= rx_x <= 1000:
            intermediate_uav = (500, 2.5)
        elif 1000 < tx_x <= 2000 and 1000 < rx_x <= 2000:
            intermediate_uav = (1500, 2.5)
        elif 2000 < tx_x <= 3000 and 2000 < rx_x <= 3000:
            intermediate_uav = (2500, 2.5)
        else:
            intermediate_uav = None

        # Check if intermediate_uav is not None
        if intermediate_uav is not None:
            time_uav = 0

        # Check if env._line_of_sight is available
        if hasattr(env, '_line_of_sight'):
            for i, pos1 in enumerate(positions_old):
                if pos1 != tx_pos and pos1 != rx_pos:
                    if env._line_of_sight(pos1, tx_pos):
                        tx_list.add(i)
                    if env._line_of_sight(pos1, rx_pos):
                        rx_list.add(i)

            # Find common positions using set intersection
            common_positions = tx_list & rx_list

            # If there are common positions, add to the group
            if common_positions:
                common_positions_list = list(common_positions)
                group0.append((tx, rx, common_positions_list))

            # Initialize the list to store times
            times = []
            time_v2v = 0
            if common_positions:
                if hasattr(env, '_calculate_throughput'):
                    # Iterate through common_positions_list and calculate times
                    for pos in common_positions_list:
                        flow1 = (tx, pos)
                        flow2 = (pos, rx)
                        time = _V2V_Time(flow1) + _V2V_Time(flow2)
                        times.append(time)

                    time_v2v = min(times)

                    # Find the index of the minimum time
                    min_time_index = times.index(min(times))

                    # Set hop_flow to the corresponding position in common_positions_list
                    hop_flow = common_positions_list[min_time_index]
        else:
            print(f"Invalid positions for tx: {tx} or rx: {rx}")

        if (time_uav > 0 and time_v2v > 0 and time_uav < time_v2v) or (time_uav > 0 and time_v2v == 0):
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            flows = [flow for flow in flows if flow != (tx, rx)]
            flows.append((tx, rx))
            # Add the hop time to time_slot
            time_slots.append(time_uav)
            print(f"UAV picked")

        elif (time_uav > 0 and time_v2v > 0 and time_uav >= time_v2v) or (time_v2v > 0 and time_uav == 0):
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            flows = [flow for flow in flows if flow != (tx, rx)]
            flows.append((tx, rx))
            # Add the hop time to time_slot
            time_slots.append(time_v2v)
            print(f"CAR {hop_flow} picked")
        else:
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            rejects.append([flow for flow in flows if flow == (tx, rx)][0])
            flows = [flow for flow in flows if flow != (tx, rx)]
            print("No UAV or CAR picked for hop")

flows_new = flows.copy()

print(flows_old)
print(rejects)
print(f"No of concluded flows are: {len(flows_old) - len(rejects)}")
print(flows)
print(time_slots)


No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
[(18, 26), (48, 36), (11, 5), (53, 16), (11, 9), (56, 7), (3, 1), (21, 36), (6, 24), (54, 55), (22, 13), (13, 41), (59, 35), (50, 53), (29, 30), (6, 46), (33, 18), (4, 58), (28, 6), (9, 11), (21, 30), (50, 43), (31, 29), (54, 40), (23, 34), (31, 56), (47, 0), (34, 53), (1, 46), (57, 22), (19, 31), (0, 27), (30, 49), (39, 27), (53, 56), (44, 46), (57, 12), (15, 33), (33, 32), (14, 45), (49, 30), (44, 54), (59, 57), (53, 0), (34, 23), (37, 19), (1

Generate Data For UAV but no-satellite

In [ ]:
import pandas as pd
import ast
import math
import numpy as np

# Initialize all variables to empty lists
flows = []
time_slots = []
los_data = []
time_slots_old = []
positions = []
positions_old = []
flows_old = []

# Load the CSV files
data_df = pd.read_csv('/content/data.csv')
los_data_df = pd.read_csv('/content/los_data.csv')
positions_data_df = pd.read_csv('/content/positions_data.csv')

# Extract tx and rx from the 'Flow' column
flows = [(int(flow.split('->')[0]), int(flow.split('->')[1])) for flow in data_df['Flow']]

# Extract time slots from the 'Time Slots' column
time_slots_old = data_df['Time Slots'].to_dict()

# Extract LOS data
los_data = los_data_df['LOS'].to_dict()

# Remove time slots greater than 10000 and corresponding data in LOS and flows
filtered_time_slots = {k: v for k, v in time_slots_old.items()}

# Keep only the flows and LOS data corresponding to the filtered time slots
filtered_flows = [flows[i] for i in filtered_time_slots.keys()]
filtered_los_data = {k: los_data[k] for k in filtered_time_slots.keys()}

# Update the original variables with the filtered data
time_slots = list(filtered_time_slots.values())
flows = filtered_flows
flows_old = flows.copy()  # Use copy() to avoid changing flows_old if flows is modified later
los_data = filtered_los_data

# Convert the 'Positions' column back to a list of tuples
# Use eval to handle NumPy objects
positions_list = [eval(pos) for pos in positions_data_df['Positions']]
positions_old = [list(pos) for pos in positions_list]
positions = [list(pos) for pos in positions_list]

def _V2V_Time(flow):
    # Calculate throughput using NumPy vectorization
    tx, rx = flow
    G0_dBi = 20  # Placeholder value for the beamforming gain
    G0 = 10 ** (G0_dBi / 10)
    theta_3dB = 30  # Placeholder value for the 3dB angle
    P_t_dBm = 40  # Transmit power
    P_t = 10**((P_t_dBm-30)/10)
    alpha_v = 2.5  # Path loss exponent
    carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
    wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
    k_v = (wavelength / (4 * math.pi)) ** alpha_v  # Constant factor for power calculations
    N0 = -134  # Noise power spectral density
    W = 2000  # Bandwidth (MHz)
    eta = 0.8  # Spectral efficiency
    Q_a = Qa  # Placeholder for the amount of data to be transmitted
    M = max_slots  # Placeholder for the number of transmissions
    T = 0.1  # Duration of one time slot

    def distance(pos1, pos2):
        return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

    g = 1  # Placeholder for channel gain
    d_s_r = distance(positions[tx], positions[rx])

    P_r_s_r = k_v * P_t * G0 * g * d_s_r ** -alpha_v
    SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps

    time_slots = math.ceil(Q_a * M / (rate))
    print(f"Time Slots: {time_slots}")
    print(rate)

    return time_slots


def _V2U_Time(tx, uav):
    G0_dBi = 20  # Placeholder value for the beamforming gain
    G0 = 10 ** (G0_dBi / 10)
    theta_3dB = 30  # Placeholder value for the 3dB angle
    P_t_dBm = 40  # Transmit power
    P_t = 10**((P_t_dBm-30)/10)
    alpha_u = 2  # Path loss exponent
    carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
    wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
    k_u = (wavelength / (4 * math.pi)) ** alpha_u  # Constant factor for power calculations
    N0 = -134  # Noise power spectral density
    W = 2000  # Bandwidth (MHz)
    eta = 0.8  # Spectral efficiency
    Q_a = Qa  # Placeholder for the amount of data to be transmitted
    M = max_slots  # Placeholder for the number of transmissions
    T = 0.1  # Duration of one time slot

    def distance(tx, uav):
        return np.cbrt((tx[0] - uav[0]) ** 2 + (tx[1] - uav[1]) ** 2 + 100 ** 2)  # Access tuple elements for subtraction

    g = 1  # Placeholder for channel gain
    d_s_r = distance(tx, uav)

    P_r_s_r = k_u * P_t * G0 * g * d_s_r ** -alpha_u

    SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps
    time_slots = math.ceil(Q_a * M / (rate))

    return time_slots

# Initialize groups outside the loop
group1 = []
group0 = []
group = []
rejects = []

# Process the flows and LOS data
for flow_index, (tx, rx) in enumerate(flows_old):
    if los_data.get(flow_index) == 0:
        time_uav = 0
        time_v2v = 0

        # Find positions for tx and rx
        tx_pos = positions[tx] if tx < len(positions_old) else None
        rx_pos = positions[rx] if rx < len(positions_old) else None

        # Check if tx_pos and rx_pos are valid
        if tx_pos and rx_pos:
            # Initialize tx_list and rx_list as sets to avoid duplicates
            tx_list = set()
            rx_list = set()

        # Check for UAV hop
        tx_x = tx_pos[0]
        rx_x = rx_pos[0]

        # Determine the intermediate UAV position
        if 0 <= tx_x <= 1000 and 0 <= rx_x <= 1000:
            intermediate_uav = (500, 2.5)
        elif 1000 < tx_x <= 2000 and 1000 < rx_x <= 2000:
            intermediate_uav = (1500, 2.5)
        elif 2000 < tx_x <= 3000 and 2000 < rx_x <= 3000:
            intermediate_uav = (2500, 2.5)
        else:
            intermediate_uav = None

        # Check if intermediate_uav is not None
        if intermediate_uav is not None:
            time_uav = _V2U_Time(tx_pos, intermediate_uav) + _V2U_Time(intermediate_uav, rx_pos)

        # Check if env._line_of_sight is available
        if hasattr(env, '_line_of_sight'):
            for i, pos1 in enumerate(positions_old):
                if pos1 != tx_pos and pos1 != rx_pos:
                    if env._line_of_sight(pos1, tx_pos):
                        tx_list.add(i)
                    if env._line_of_sight(pos1, rx_pos):
                        rx_list.add(i)

            # Find common positions using set intersection
            common_positions = tx_list & rx_list

            # If there are common positions, add to the group
            if common_positions:
                common_positions_list = list(common_positions)
                group0.append((tx, rx, common_positions_list))

            # Initialize the list to store times
            times = []
            time_v2v = 0
            if common_positions:
                if hasattr(env, '_calculate_throughput'):
                    # Iterate through common_positions_list and calculate times
                    for pos in common_positions_list:
                        flow1 = (tx, pos)
                        flow2 = (pos, rx)
                        time = _V2V_Time(flow1) + _V2V_Time(flow2)
                        times.append(time)

                    time_v2v = min(times)

                    # Find the index of the minimum time
                    min_time_index = times.index(min(times))

                    # Set hop_flow to the corresponding position in common_positions_list
                    hop_flow = common_positions_list[min_time_index]
        else:
            print(f"Invalid positions for tx: {tx} or rx: {rx}")

        if (time_uav > 0 and time_v2v > 0 and time_uav < time_v2v) or (time_uav > 0 and time_v2v == 0):
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            flows = [flow for flow in flows if flow != (tx, rx)]
            flows.append((tx, rx))
            # Add the hop time to time_slot
            time_slots.append(time_uav)
            print(f"UAV picked")

        elif (time_uav > 0 and time_v2v > 0 and time_uav >= time_v2v) or (time_v2v > 0 and time_uav == 0):
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            flows = [flow for flow in flows if flow != (tx, rx)]
            flows.append((tx, rx))
            # Add the hop time to time_slot
            time_slots.append(time_v2v)
            print(f"CAR {hop_flow} picked")
        else:
            # Find the index corresponding to (tx, rx) in flows
            slot = flows.index((tx, rx))
            # Remove the element at the index 'slot' from time_slots
            time_slots.pop(slot)
            # Update the flows list
            rejects.append([flow for flow in flows if flow == (tx, rx)][0])
            flows = [flow for flow in flows if flow != (tx, rx)]
            print("No UAV or CAR picked for hop")

flows_new = flows.copy()
time_new = time_slots.copy()

print(flows_old)
print(rejects)
print(f"No of concluded flows are: {len(flows_old) - len(rejects)}")
print(flows)
print(time_slots)


No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
UAV picked
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
UAV picked
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
UAV picked
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
No UAV or CAR picked for hop
UAV picked
No UAV or CAR picked for hop
No UAV or CA

Generate Data for uav and satellite

In [ ]:
# import pandas as pd
# import ast
# import math
# import numpy as np

# # Initialize all variables to empty lists
# flows = []
# time_slots = []
# los_data = []
# time_slots_old = []
# positions = []
# positions_old = []
# flows_old = []

# # Load the CSV files
# data_df = pd.read_csv('/content/data.csv')
# los_data_df = pd.read_csv('/content/los_data.csv')
# positions_data_df = pd.read_csv('/content/positions_data.csv')

# # Extract tx and rx from the 'Flow' column
# flows = [(int(flow.split('->')[0]), int(flow.split('->')[1])) for flow in data_df['Flow']]

# # Extract time slots from the 'Time Slots' column
# time_slots_old = data_df['Time Slots'].to_dict()

# # Extract LOS data
# los_data = los_data_df['LOS'].to_dict()

# # Remove time slots greater than 10000 and corresponding data in LOS and flows
# filtered_time_slots = {k: v for k, v in time_slots_old.items()}

# # Keep only the flows and LOS data corresponding to the filtered time slots
# filtered_flows = [flows[i] for i in filtered_time_slots.keys()]
# filtered_los_data = {k: los_data[k] for k in filtered_time_slots.keys()}

# # Update the original variables with the filtered data
# time_slots = list(filtered_time_slots.values())
# flows = filtered_flows
# flows_old = flows.copy()  # Use copy() to avoid changing flows_old if flows is modified later
# los_data = filtered_los_data

# # Convert the 'Positions' column back to a list of tuples
# # Use eval to handle NumPy objects
# positions_list = [eval(pos) for pos in positions_data_df['Positions']]
# positions_old = [list(pos) for pos in positions_list]
# positions = [list(pos) for pos in positions_list]

# Qa = 100  # Placeholder for the amount of data to be transmitted (Mb)
# max_slots = 1  # Placeholder for the number of transmission attempts

# def _V2V_Time(flow):
#     # Calculate throughput using NumPy vectorization
#     tx, rx = flow
#     G0_dBi = 20  # Placeholder value for the beamforming gain
#     G0 = 10 ** (G0_dBi / 10)
#     theta_3dB = 30  # Placeholder value for the 3dB angle
#     P_t_dBm = 40  # Transmit power
#     P_t = 10**((P_t_dBm-30)/10)
#     alpha_v = 2.5  # Path loss exponent
#     carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
#     wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
#     k_v = (wavelength / (4 * math.pi)) ** alpha_v  # Constant factor for power calculations
#     N0 = -134  # Noise power spectral density
#     W = 2000  # Bandwidth (MHz)
#     eta = 0.8  # Spectral efficiency
#     Q_a = Qa  # Placeholder for the amount of data to be transmitted
#     M = max_slots  # Placeholder for the number of transmissions
#     T = 0.1  # Duration of one time slot

#     def distance(pos1, pos2):
#         return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

#     g = 1  # Placeholder for channel gain
#     d_s_r = distance(positions[tx], positions[rx])

#     P_r_s_r = k_v * P_t * G0 * g * d_s_r ** -alpha_v
#     SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
#     rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps

#     time_slots = math.ceil(Q_a * M / (rate))
#     print(f"Time Slots: {time_slots}")
#     print(rate)

#     return time_slots


# def _V2U_Time(tx, uav):
#     G0_dBi = 20  # Placeholder value for the beamforming gain
#     G0 = 10 ** (G0_dBi / 10)
#     theta_3dB = 30  # Placeholder value for the 3dB angle
#     P_t_dBm = 40  # Transmit power
#     P_t = 10**((P_t_dBm-30)/10)
#     alpha_u = 2  # Path loss exponent
#     carrier_frequency = 30 * 10**9  # Carrier frequency in Hz
#     wavelength = (3 * 10**8) / carrier_frequency  # Wavelength in meters
#     k_u = (wavelength / (4 * math.pi)) ** alpha_u  # Constant factor for power calculations
#     N0 = -134  # Noise power spectral density
#     W = 2000  # Bandwidth (MHz)
#     eta = 0.8  # Spectral efficiency
#     Q_a = Qa # Placeholder for the amount of data to be transmitted
#     M = max_slots  # Placeholder for the number of transmissions
#     T = 0.1  # Duration of one time slot

#     def distance(tx, uav):
#         return np.cbrt((tx[0] - uav[0]) ** 2 + (tx[1] - uav[1]) ** 2 + 100 ** 2)  # Access tuple elements for subtraction

#     g = 1  # Placeholder for channel gain
#     d_s_r = distance(tx, uav)

#     P_r_s_r = k_u * P_t * G0 * g * d_s_r ** -alpha_u

#     SNR = P_r_s_r / (10 ** ((N0 +10*np.log10(W)-30) / 10))
#     rate = eta * W * np.log2(1 + SNR) * 10**-3  # Gbps
#     time_slots = math.ceil(Q_a * M / (rate))

#     return time_slots

# # --- Satellite Time Function ---
# def satellite_time(tx_pos, rx_pos):
#     G_es_dBi = 48  # Earth Station gain
#     P_es_dBm = 44  # Earth Station power
#     G_es = 10 ** (G_es_dBi / 10)
#     P_es = 10 ** ((P_es_dBm - 30) / 10)

#     G_sat_tx_dBi = 38.5  # Satellite antenna gain
#     G_ship_rx_dBi = 2    # Ship (vehicle) receiver gain
#     P_sat_dBm = 40       # Satellite power
#     G_sat_tx = 10 ** (G_sat_tx_dBi / 10)
#     G_ship_rx = 10 ** (G_ship_rx_dBi / 10)
#     P_sat = 10 ** ((P_sat_dBm - 30) / 10)

#     alpha_sat = 2
#     carrier_frequency = 30e9
#     wavelength = 3e8 / carrier_frequency
#     k_sat = (wavelength / (4 * math.pi)) ** alpha_sat

#     N0_dBm_Hz = -174
#     BW = 250e6  # 250 MHz
#     N0_total_dBm = N0_dBm_Hz + 10 * np.log10(BW)
#     N0 = 10 ** ((N0_total_dBm - 30) / 10)

#     eta = 0.8
#     T = 0.1  # Time slot duration

#     h_sat = 1200e3
#     d = math.sqrt((tx_pos[0] - rx_pos[0]) ** 2 + (tx_pos[1] - rx_pos[1]) ** 2)
#     R = math.sqrt(h_sat ** 2 + d ** 2)

#     g = 1
#     P_r_uplink = k_sat * P_es * G_es * G_sat_tx * g * R ** -alpha_sat
#     SNR_uplink = P_r_uplink / N0
#     rate_uplink = eta * BW * np.log2(1 + SNR_uplink) * 1e-6  # Mbps

#     P_r_downlink = k_sat * P_sat * G_sat_tx * G_ship_rx * g * R ** -alpha_sat
#     SNR_downlink = P_r_downlink / N0
#     rate_downlink = eta * BW * np.log2(1 + SNR_downlink) * 1e-6  # Mbps

#     rate = min(rate_uplink, rate_downlink)
#     required_time_slots = math.ceil(Qa * max_slots / rate)
#     return required_time_slots

# # Initialize groups outside the loop
# group1 = []
# group0 = []
# group = []
# rejects = []

# # Process the flows and LOS data
# for flow_index, (tx, rx) in enumerate(flows_old):
#     if los_data.get(flow_index) == 0:
#         time_uav = 0
#         time_v2v = 0

#         # Find positions for tx and rx
#         tx_pos = positions[tx] if tx < len(positions_old) else None
#         rx_pos = positions[rx] if rx < len(positions_old) else None

#         # Check if tx_pos and rx_pos are valid
#         if tx_pos and rx_pos:
#             # Initialize tx_list and rx_list as sets to avoid duplicates
#             tx_list = set()
#             rx_list = set()

#         # Check for UAV hop
#         tx_x = tx_pos[0]
#         rx_x = rx_pos[0]

#         # Determine the intermediate UAV position
#         if 0 <= tx_x <= 1000 and 0 <= rx_x <= 1000:
#             intermediate_uav = (500, 2.5)
#         elif 1000 < tx_x <= 2000 and 1000 < rx_x <= 2000:
#             intermediate_uav = (1000, 2.5)
#         elif 2000 < tx_x <= 3000 and 2000 < rx_x <= 3000:
#             intermediate_uav = (2500, 2.5)
#         else:
#             intermediate_uav = None

#         # Check if intermediate_uav is not None
#         if intermediate_uav is not None:
#             time_uav = _V2U_Time(tx_pos, intermediate_uav) + _V2U_Time(intermediate_uav, rx_pos)

#         # Check if env._line_of_sight is available
#         if hasattr(env, '_line_of_sight'):
#             for i, pos1 in enumerate(positions_old):
#                 if pos1 != tx_pos and pos1 != rx_pos:
#                     if env._line_of_sight(pos1, tx_pos):
#                         tx_list.add(i)
#                     if env._line_of_sight(pos1, rx_pos):
#                         rx_list.add(i)

#             # Find common positions using set intersection
#             common_positions = tx_list & rx_list

#             # If there are common positions, add to the group
#             if common_positions:
#                 common_positions_list = list(common_positions)
#                 group0.append((tx, rx, common_positions_list))

#             # Initialize the list to store times
#             times = []
#             time_v2v = 0
#             if common_positions:
#                 if hasattr(env, '_calculate_throughput'):
#                     # Iterate through common_positions_list and calculate times
#                     for pos in common_positions_list:
#                         flow1 = (tx, pos)
#                         flow2 = (pos, rx)
#                         time = _V2V_Time(flow1) + _V2V_Time(flow2)
#                         times.append(time)

#                     time_v2v = min(times)

#                     # Find the index of the minimum time
#                     min_time_index = times.index(min(times))

#                     # Set hop_flow to the corresponding position in common_positions_list
#                     hop_flow = common_positions_list[min_time_index]
#         else:
#           #print(f"Invalid positions for tx: {tx} or rx: {rx}")
#             if tx_pos is not None and rx_pos is not None:
#               time_sat = satellite_time(tx_pos, rx_pos)
#               if time_sat > 0:
#                 slot = flows.index((tx, rx))
#                 time_slots.pop(slot)
#                 flows = [flow for flow in flows if flow != (tx, rx)]
#                 flows.append((tx, rx))
#                 time_slots.append(time_sat)
#                 print((tx_pos[0], rx_pos[0]))
#                 print("Satellite link picked")
#         else:
#             slot = flows.index((tx, rx))
#             time_slots.pop(slot)
#             rejects.append([flow for flow in flows if flow == (tx, rx)][0])
#             flows = [flow for flow in flows if flow != (tx, rx)]
#             print("No UAV, CAR, or Satellite path available")

#         if (time_uav > 0 and time_v2v > 0 and time_uav < time_v2v) or (time_uav > 0 and time_v2v == 0):
#             # Find the index corresponding to (tx, rx) in flows
#             slot = flows.index((tx, rx))
#             # Remove the element at the index 'slot' from time_slots
#             time_slots.pop(slot)
#             # Update the flows list
#             flows = [flow for flow in flows if flow != (tx, rx)]
#             flows.append((tx, rx))
#             # Add the hop time to time_slot
#             time_slots.append(time_uav)
#             print(f"UAV picked")

#         elif (time_uav > 0 and time_v2v > 0 and time_uav >= time_v2v) or (time_v2v > 0 and time_uav == 0):
#             # Find the index corresponding to (tx, rx) in flows
#             slot = flows.index((tx, rx))
#             # Remove the element at the index 'slot' from time_slots
#             time_slots.pop(slot)
#             # Update the flows list
#             flows = [flow for flow in flows if flow != (tx, rx)]
#             flows.append((tx, rx))
#             # Add the hop time to time_slot
#             time_slots.append(time_v2v)
#             print(f"CAR {hop_flow} picked")
#         else:
#           time_sat = satellite_time(tx_pos, rx_pos)
#           if time_sat > 0:
#                slot = flows.index((tx, rx))
#                time_slots.pop(slot)
#                flows = [flow for flow in flows if flow != (tx, rx)]
#                flows.append((tx, rx))
#                time_slots.append(time_sat)
#                print((tx_x,rx_x))
#                print("Satellite link picked")
# else:
#     slot = flows.index((tx, rx))
#     time_slots.pop(slot)
#     rejects.append([flow for flow in flows if flow == (tx, rx)][0])
#     flows = [flow for flow in flows if flow != (tx, rx)]
#     print("No UAV, CAR, or Satellite path available")





# flows_new = flows.copy()
# time_new = time_slots.copy()

# print(flows_old)
# print(rejects)
# print(f"No of concluded flows are: {len(flows_old) - len(rejects)}")
# print(flows)
# print(time_slots)
import pandas as pd
import ast
import math
import numpy as np

# Initialize all variables to empty lists
flows = []
time_slots = []
los_data = []
time_slots_old = []
positions = []
positions_old = []
flows_old = []

# Load the CSV files
data_df = pd.read_csv('/content/data.csv')
los_data_df = pd.read_csv('/content/los_data.csv')
positions_data_df = pd.read_csv('/content/positions_data.csv')

# Extract tx and rx from the 'Flow' column
flows = [(int(flow.split('->')[0]), int(flow.split('->')[1])) for flow in data_df['Flow']]

# Extract time slots from the 'Time Slots' column
time_slots_old = data_df['Time Slots'].to_dict()

# Extract LOS data
los_data = los_data_df['LOS'].to_dict()

# Keep only the flows and LOS data corresponding to the filtered time slots
filtered_time_slots = {k: v for k, v in time_slots_old.items()}
filtered_flows = [flows[i] for i in filtered_time_slots.keys()]
filtered_los_data = {k: los_data[k] for k in filtered_time_slots.keys()}

# Update the original variables with the filtered data
time_slots = list(filtered_time_slots.values())
flows = filtered_flows
flows_old = flows.copy()
los_data = filtered_los_data

# Convert the 'Positions' column back to a list of tuples
positions_list = [eval(pos) for pos in positions_data_df['Positions']]
positions_old = [list(pos) for pos in positions_list]
positions = [list(pos) for pos in positions_list]

Qa = 0.5

def _V2V_Time(flow):
    tx, rx = flow
    G0_dBi = 20
    G0 = 10 ** (G0_dBi / 10)
    alpha_v = 2.5
    carrier_frequency = 30 * 10**9
    wavelength = (3 * 10**8) / carrier_frequency
    k_v = (wavelength / (4 * math.pi)) ** alpha_v
    N0 = -134
    W = 2000
    eta = 0.8
    Q_a = Qa
    M = max_slots

    def distance(pos1, pos2):
        return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

    g = 1
    d_s_r = distance(positions[tx], positions[rx])
    P_t_dBm = 40
    P_t = 10 ** ((P_t_dBm - 30) / 10)
    P_r_s_r = k_v * P_t * G0 * g * d_s_r ** -alpha_v
    SNR = P_r_s_r / (10 ** ((N0 + 10 * np.log10(W) - 30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 1e-3

    return math.ceil(Q_a * M / rate)

def _V2U_Time(tx, uav):
    G0_dBi = 20
    G0 = 10 ** (G0_dBi / 10)
    alpha_u = 2
    carrier_frequency = 30 * 10**9
    wavelength = (3 * 10**8) / carrier_frequency
    k_u = (wavelength / (4 * math.pi)) ** alpha_u
    N0 = -134
    W = 2000
    eta = 0.8
    Q_a = Qa
    M = max_slots

    def distance(tx, uav):
        return np.cbrt((tx[0] - uav[0]) ** 2 + (tx[1] - uav[1]) ** 2 + 100 ** 2)

    g = 1
    d_s_r = distance(tx, uav)
    P_t_dBm = 40
    P_t = 10 ** ((P_t_dBm - 30) / 10)
    P_r_s_r = k_u * P_t * G0 * g * d_s_r ** -alpha_u
    SNR = P_r_s_r / (10 ** ((N0 + 10 * np.log10(W) - 30) / 10))
    rate = eta * W * np.log2(1 + SNR) * 1e-3

    return math.ceil(Q_a * M / rate)

def satellite_time(tx_pos, rx_pos):
    G_es_dBi = 48  #antenna gain of earth station(ES)
    P_es_dBm = 44
    G_es = 10 ** (G_es_dBi / 10)
    P_es = 10 ** ((P_es_dBm - 30) / 10)

    G_sat_tx_dBi = 38.5
    G_ship_rx_dBi = 2
    P_sat_dBm = 40
    G_sat_tx = 10 ** (G_sat_tx_dBi / 10)
    G_ship_rx = 10 ** (G_ship_rx_dBi / 10)
    P_sat = 10 ** ((P_sat_dBm - 30) / 10)

    alpha_sat = 2
    carrier_frequency = 30e9
    wavelength = 3e8 / carrier_frequency
    k_sat = (wavelength / (4 * math.pi)) ** alpha_sat

    N0_dBm_Hz = -174
    BW = 2000e6
    N0_total_dBm = N0_dBm_Hz + 10 * np.log10(BW)
    N0 = 10 ** ((N0_total_dBm - 30) / 10)

    eta = 0.8
    T=0.1
    h_sat = 1200e3
    d = math.sqrt((tx_pos[0] - rx_pos[0]) ** 2 + (tx_pos[1] - rx_pos[1]) ** 2)
    R = math.sqrt(h_sat ** 2 +(tx_pos[0] - 1500) ** 2 + (tx_pos[1]) ** 2)
    M=max_slots
    g = 1
    P_r_uplink = k_sat * P_es * G_es * G_sat_tx * g * R ** -alpha_sat
    SNR_uplink = P_r_uplink / N0
    rate_uplink = eta * BW * np.log2(1 + SNR_uplink)*1e-6
    R1 = math.sqrt(h_sat ** 2 +(rx_pos[0] - 1500) ** 2 + (rx_pos[1]) ** 2)
    P_r_downlink = k_sat * P_sat * G_sat_tx * G_ship_rx * g * R1 ** -alpha_sat
    SNR_downlink = P_r_downlink / N0
    rate_downlink = eta * BW * np.log2(1 + SNR_downlink) *1e-6

    rate = min(rate_uplink, rate_downlink)
    return math.ceil(Qa * M / rate)

group0 = []
rejects = []

for flow_index, (tx, rx) in enumerate(flows_old):
    if los_data.get(flow_index) == 0:
        time_uav = 0
        time_v2v = 0

        tx_pos = positions[tx] if tx < len(positions_old) else None
        rx_pos = positions[rx] if rx < len(positions_old) else None

        if tx_pos and rx_pos:
            tx_x = tx_pos[0]
            rx_x = rx_pos[0]

            if 0 <= tx_x <= 1000 and 0 <= rx_x <= 1000:
                intermediate_uav = (500, 3.75)
            elif 1000 < tx_x <= 2000 and 1000 < rx_x <= 2000:
                intermediate_uav = (1500, 3.75)
            elif 2000 < tx_x <= 3000 and 2000 < rx_x <= 3000:
                intermediate_uav = (2500, 3.75)
            else:
                intermediate_uav = None

            if intermediate_uav:
                time_uav = _V2U_Time(tx_pos, intermediate_uav) + _V2U_Time(intermediate_uav, rx_pos)

            if hasattr(env, '_line_of_sight'):
                tx_list = set()
                rx_list = set()
                for i, pos1 in enumerate(positions_old):
                    if pos1 != tx_pos and pos1 != rx_pos:
                        if env._line_of_sight(pos1, tx_pos):
                            tx_list.add(i)
                        if env._line_of_sight(pos1, rx_pos):
                            rx_list.add(i)
                common_positions = tx_list & rx_list

                if common_positions:
                    common_positions_list = list(common_positions)
                    group0.append((tx, rx, common_positions_list))
                    times = []
                    for pos in common_positions_list:
                        time = _V2V_Time((tx, pos)) + _V2V_Time((pos, rx))
                        times.append(time)
                    time_v2v = min(times)
                    min_time_index = times.index(min(times))
                    hop_flow = common_positions_list[min_time_index]

        # Satellite fallback if both UAV and V2V failed
        if time_uav == 0 and time_v2v == 0:
            time_sat = satellite_time(tx_pos, rx_pos)
            if time_sat > 0:
                slot = flows.index((tx, rx))
                time_slots.pop(slot)
                flows.remove((tx, rx))
                flows.append((tx, rx))
                time_slots.append(time_sat)
                print("Satellite link picked"," tx:",tx_x," rx:",rx_x,time_sat)
            else:
                slot = flows.index((tx, rx))
                time_slots.pop(slot)
                rejects.append((tx, rx))
                flows.remove((tx, rx))
                print("No UAV, CAR, or Satellite path available"," tx:",tx_x," rx:",rx_x)

        elif (time_uav > 0 and time_v2v > 0 and time_uav < time_v2v) or (time_uav > 0 and time_v2v == 0):
            slot = flows.index((tx, rx))
            time_slots.pop(slot)
            flows.remove((tx, rx))
            flows.append((tx, rx))
            time_slots.append(time_uav)
            print("UAV picked"," tx:",tx_x," rx:",rx_x)

        elif (time_uav > 0 and time_v2v > 0 and time_uav >= time_v2v) or (time_v2v > 0 and time_uav == 0):
            slot = flows.index((tx, rx))
            time_slots.pop(slot)
            flows.remove((tx, rx))
            flows.append((tx, rx))
            time_slots.append(time_v2v)
            print(f"CAR {hop_flow} picked"," tx:",tx_x," rx:",rx_x)

flows_new = flows.copy()
time_new = time_slots.copy()
print(flows_old)
print(rejects)
print(f"No of concluded flows are: {len(flows_old) - len(rejects)}")
print(flows)
print(time_slots)


UAV picked  tx: 196.18204302101498  rx: 3.2990047907201303
Satellite link picked  tx: 1346.2068501228152  rx: 348.96840040234054 439
UAV picked  tx: 362.1670498029911  rx: 901.0222563566576
Satellite link picked  tx: 590.7512316240511  rx: 1010.9523932679849 439
UAV picked  tx: 420.8562266597682  rx: 590.7512316240511
Satellite link picked  tx: 832.8988850398517  rx: 1846.1533946671507 439
Satellite link picked  tx: 690.7006241390045  rx: 1571.6262016569117 439
UAV picked  tx: 1010.9523932679849  rx: 1904.066446738671
Satellite link picked  tx: 393.7306851296971  rx: 1904.066446738671 439
Satellite link picked  tx: 690.7006241390045  rx: 1453.967946804598 439
Satellite link picked  tx: 75.65947839313093  rx: 1944.6735340546516 439
Satellite link picked  tx: 1543.2257335701452  rx: 196.18204302101498 439
UAV picked  tx: 1878.4252486892337  rx: 1056.479931296582
UAV picked  tx: 1878.4252486892337  rx: 1571.6262016569117
UAV picked  tx: 383.29846392854483  rx: 890.2986518635416
Satellite 

JRDS

In [ ]:
def generate_group(V, E):
    G = set()

    while V:
        # Obtain vertex with minimum degree
        v = min(V, key=lambda x: len([e for e in E if x in e]))
        G.add(v)

        # Obtain flows connected to v and update the sets V and E
        connected_flows = {v_prime for v_prime in V if (v, v_prime) in E or (v_prime, v) in E}
        V = V - connected_flows - {v}
        E = E - {(v, v_prime) for v_prime in connected_flows} - {(v_prime, v) for v_prime in connected_flows}

    return G

def flow_grouping_algorithm(V, E):
    G_grouped = []

    while V:
        G = generate_group(V, E)
        V = V - G
        E = E - {(v, v_prime) for v in G for v_prime in G if (v, v_prime) in E or (v_prime, v) in E}
        G_grouped.append(G)

    return G_grouped

def DGroup(Vd, Ed, G):
    Edr = {(v1, v2) for v1 in Vd for v2 in G if (v1, v2) in Ed or (v2, v1) in Ed}
    E = Ed | Edr
    DG = generate_group(Vd, E)
    return DG

def RGroup(Vr, Rg, G):
    RG = set()

    for v in Vr:
        for r in Rg[v]:
            if all((r, g) not in G and (g, r) not in G for g in G | RG):
                RG.add(r)
                break

    return RG






In [ ]:
def JRDS_algorithm(Vd, Ed, Vr, Rg, M, Qg, N, time_slots):
    G = set()
    G_grouped = []
    state = np.zeros(len(flows))
    Vr = set()

    # Initialize slots with Qg * M for each element (size 20)
    slots = time_slots.copy()  # Use a copy to avoid modifying the original slots array

    # Step 1: Run the flow grouping algorithm to generate groups of flows
    grouped_flows = flow_grouping_algorithm(Vd.copy(), Ed.copy())
    current_group_index = 0  # Index to keep track of which group to select from

    # Step 2: Initialize the first group
    if grouped_flows:
        G = grouped_flows[current_group_index]
        current_group_index += 1 if not G else current_group_index  # Increment index only if the group is empty
    else:
        return G_grouped, 0, 0  # No groups to schedule

    for m in range(1, M + 1):

        print(f"Time slot: {m}")
        print("Current G: ", G)
        print("Slots: ", slots)

        flows_to_remove = set()

        # Process each flow in the current group
        for flow in list(G):  # Convert to list to safely modify G while iterating
            if 0 < flow <= len(slots):  # Ensure flow index is valid
                slots[flow-1] -= 1  # Decrement the time slot for this flow
                state[flow-1] = 1  # Mark flow as active

                if slots[flow-1] <= 0:  # If time slots run out, mark the flow as completed
                    state[flow-1] = 2
                    flows_to_remove.add(flow)  # Collect flows to remove

        # Remove completed flows from G
        G -= flows_to_remove

        # Step 3: Add all valid candidates from the next group (grouped_flows)
        if current_group_index < len(grouped_flows) - 1:
            next_group = grouped_flows[current_group_index  + 1]
            for candidate_flow in list(next_group):  # Use list to modify while iterating
                # Add the flow to G if it has no adjacency with any flow in the current G
                if all(arr[candidate_flow-1][g-1] == 0 for g in G):
                    G.add(candidate_flow)
                    next_group.remove(candidate_flow)  # Remove from next_group after adding
                    print(f"Added new flow {candidate_flow} to G")

            # Increment current_group_index only if the current group is empty
            if not next_group:
                print(current_group_index)
                current_group_index += 1

        # Break if all flows are completed and there are no more groups
        if not G and current_group_index >= len(grouped_flows) - 1:
            break

    # Compute utility
    U = Qg * N / (m * 0.1)

    return G_grouped, m, U


In [ ]:
import pandas as pd
import ast  # For converting string representation of lists back to list format


flows = flows_new  # Assuming flows_new is defined

# Get the number of flows
num_flows = len(flows)

# Initialize arr as a 2D array of zeros (size num_flows x num_flows)
arr = [[0] * num_flows for _ in range(num_flows)]

# Populate arr to track contending flows
for i, flow_i in enumerate(flows):
    for j, flow_j in enumerate(flows):
        if i != j:
            tx_i, rx_i = flow_i
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                arr[i][j] = 1  # Set arr[i][j] to 1 if any condition matches

# Create V as a set numbered from 1 to num_flows
V = set(range(1, num_flows + 1))

# Create E as the set of tuples (i+1, j+1) corresponding to 1 in arr
E = set()
for i in range(num_flows):
    for j in range(num_flows):
        if arr[i][j] == 1:
            E.add((i + 1, j + 1))  # i+1, j+1 because vertices are 1-based

# V is the set of vertices and E is the set of edges
print("Vertices (V):", V)
print("Edges (E):", E)

# Running Algorithm 1: Flow-Grouping Algorithm
grouped_flows = flow_grouping_algorithm(V.copy(), E.copy())

print("Grouped Flows:", grouped_flows)


# Sample random inputs for the JRDS Algorithm (Algorithm 3)
Vd = V  # Direct flows
Vr = {} # Relay flows
Ed = E  # Edges between direct flows
Rg = {}  # Relay group for each relay flow
M = max_slots  # Maximum number of time slots
Qg = Qa  # Throughput factor
N = len(flows)

# Define time_Slots
durations = time_slots
# Calculate rate for each element in durations
rate = [Qg * M / duration for duration in durations]
print(rate)

# Running Algorithm 3: JRDS Algorithm
scheduled_flows, total_slots, throughput = JRDS_algorithm(Vd.copy(), Ed.copy(), Vr.copy(), Rg.copy(), M, Qg, N, time_slots)

grouped_flows, scheduled_flows, total_slots, throughput


flows = flows_new
time_slots = time_new

Vertices (V): {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80}
Edges (E): {(19, 46), (67, 13), (70, 64), (42, 57), (48, 36), (36, 16), (62, 35), (3, 13), (47, 80), (54, 31), (56, 28), (76, 61), (68, 2), (32, 12), (80, 47), (52, 79), (46, 20), (49, 16), (70, 48), (7, 19), (27, 52), (30, 48), (59, 66), (24, 44), (55, 77), (2, 50), (16, 49), (40, 34), (58, 15), (68, 50), (65, 8), (2, 68), (63, 54), (66, 16), (33, 22), (14, 51), (16, 70), (77, 55), (36, 48), (47, 57), (7, 30), (57, 34), (25, 54), (56, 32), (60, 14), (60, 23), (26, 55), (13, 3), (43, 71), (43, 16), (13, 67), (26, 64), (38, 66), (49, 66), (79, 12), (21, 59), (13, 76), (44, 24), (62, 78), (61, 58), (16, 35), (17, 64), (45, 80), (76, 3), (5, 25), (41, 12), (72, 45), (3

Brute Force

In [ ]:
from ortools.sat.python import cp_model
import numpy as np

class ExactSchedulingSolver:
    def __init__(self, durations, adjacency_matrix):
        self.durations = durations
        self.adjacency_matrix = adjacency_matrix
        self.num_flows = len(durations)
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.variables = {}
        self.bool_vars = {}  # Dictionary to store Boolean variables

    def create_variables(self):
        # Create a variable for each flow to represent its starting time
        for i in range(self.num_flows):
            self.variables[i] = self.model.NewIntVar(0, sum(self.durations), f'start_time_{i}')

    def add_constraints(self):
        # Ensure non-overlapping schedules for adjacent flows
        for i in range(self.num_flows):
            for j in range(i + 1, self.num_flows):
                if self.adjacency_matrix[i, j] == 1:
                    # Create Boolean variables for precedence
                    self.bool_vars[(i, j)] = self.model.NewBoolVar(f'precedence_{i}_{j}')
                    self.bool_vars[(j, i)] = self.model.NewBoolVar(f'precedence_{j}_{i}')
                    self.model.Add(self.variables[i] + self.durations[i] <= self.variables[j]).OnlyEnforceIf(self.bool_vars[(i, j)])
                    self.model.Add(self.variables[j] + self.durations[j] <= self.variables[i]).OnlyEnforceIf(self.bool_vars[(j, i)])
                    # Enforce either i precedes j or j precedes i
                    self.model.AddBoolOr([self.bool_vars[(i, j)], self.bool_vars[(j, i)]])

    def set_objective(self):
        # Minimize the total number of time slots used
        makespan = self.model.NewIntVar(0, sum(self.durations), 'makespan')
        self.model.AddMaxEquality(makespan, [self.variables[i] + self.durations[i] for i in range(self.num_flows)])
        self.model.Minimize(makespan)

    def solve(self):
        self.create_variables()
        self.add_constraints()
        self.set_objective()
        status = self.solver.Solve(self.model)
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print(f'Optimal Schedule Found with Total Time Slots: {self.solver.ObjectiveValue()}')
            schedule = {}
            for i in range(self.num_flows):
                start_time = self.solver.Value(self.variables[i])
                schedule[i] = (start_time, start_time + self.durations[i])
            return schedule
        else:
            print('No feasible solution found.')
            return None

# Create adjacency matrix
num_flows = len(flows)
adjacency_matrix = np.zeros((num_flows, num_flows), dtype=int)
for i, flow_i in enumerate(flows):
    tx_i, rx_i = flow_i
    for j, flow_j in enumerate(flows):
        if i != j:
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                adjacency_matrix[i, j] = 1

# Initialize and solve the exact scheduling problem
solver = ExactSchedulingSolver(durations, adjacency_matrix)
schedule = solver.solve()
print(schedule)

flows = flows_new

Optimal Schedule Found with Total Time Slots: 332.0
{0: (56, 114), 1: (58, 106), 2: (0, 29), 3: (0, 32), 4: (55, 100), 5: (56, 94), 6: (59, 96), 7: (0, 37), 8: (58, 116), 9: (0, 56), 10: (111, 166), 11: (0, 58), 12: (0, 58), 13: (55, 110), 14: (0, 58), 15: (167, 225), 16: (114, 171), 17: (0, 56), 18: (0, 55), 19: (170, 225), 20: (177, 235), 21: (100, 156), 22: (0, 55), 23: (56, 111), 24: (0, 57), 25: (110, 167), 26: (56, 113), 27: (0, 57), 28: (0, 57), 29: (58, 115), 30: (57, 115), 31: (57, 111), 32: (166, 221), 33: (113, 170), 34: (0, 56), 35: (0, 58), 36: (111, 169), 37: (57, 115), 38: (0, 55), 39: (0, 56), 40: (278, 332), 41: (0, 56), 42: (0, 57), 43: (56, 114), 44: (0, 58), 45: (0, 58), 46: (32, 91), 47: (166, 221), 48: (117, 177), 49: (58, 117), 50: (60, 118), 51: (111, 167), 52: (0, 59), 53: (156, 213), 54: (91, 150), 55: (0, 55), 56: (56, 112), 57: (0, 59), 58: (0, 58), 59: (91, 149), 60: (0, 55), 61: (58, 118), 62: (0, 58), 63: (58, 117), 64: (0, 57), 65: (0, 55), 66: (56, 111)

Constraint Programming with UAV

In [ ]:
!pip install ortools

import pandas as pd
from ortools.sat.python import cp_model
import numpy as np
import ast # Import the ast module

num_flows = len(flows)
# Create adjacency matrix
adjacency_matrix = np.zeros((num_flows, num_flows), dtype=int)
for i, flow_i in enumerate(flows):
    tx_i, rx_i = flow_i
    for j, flow_j in enumerate(flows):
        if i != j:
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                adjacency_matrix[i, j] = 12

# Create E as the set of tuples (i+1, j+1) corresponding to 1 in arr
E = set()
for i in range(num_flows):
    for j in range(num_flows):
        if adjacency_matrix[i][j] == 1:
            E.add((i + 1, j + 1))  # i+1, j+1 because vertices are 1-based

print(E)

durations = time_slots
print(durations)

def schedule_flows(durations, adjacency_matrix):
    model = cp_model.CpModel()
    num_flows = len(durations)

    # Create variables for the start times of each flow
    start_times = [model.NewIntVar(0, sum(durations), f'start_{i}') for i in range(num_flows)]

    # Create variable for the total number of time slots
    total_time = model.NewIntVar(0, sum(durations), 'total_time')

    # Add adjacency constraints
    for i in range(num_flows):
        for j in range(i + 1, num_flows):
            if adjacency_matrix[i][j] == 1:
                # Introduce Boolean variables to represent the two alternative conditions
                b1 = model.NewBoolVar(f'b1_{i}_{j}')
                b2 = model.NewBoolVar(f'b2_{i}_{j}')

                # Enforce that either flow i finishes before j starts or vice versa
                model.Add(start_times[i] + durations[i] <= start_times[j]).OnlyEnforceIf(b1)
                model.Add(start_times[j] + durations[j] <= start_times[i]).OnlyEnforceIf(b2)
                # Ensure that at least one of the conditions is true
                model.AddBoolOr([b1, b2])

    # Ensure total_time is the maximum of the end times of all flows
    for i in range(num_flows):
        model.Add(total_time >= start_times[i] + durations[i])

    # Minimize the total number of time slots
    model.Minimize(total_time)

    # Create the solver and solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Minimal number of time slots: {solver.Value(total_time)}')
        for i in range(num_flows):
            print(f'Flow {i} starts at {solver.Value(start_times[i])}')
        # Return the total time
        return solver.Value(total_time) # Return the calculated total_time
    else:
        print('No solution found.')
        return None # Return None if no solution is found

total_time_result = schedule_flows(durations, adjacency_matrix) # Store the returned value

flows = flows_new

# Calculate throughput
Data = 0
Qa = 0.1
for t in time_slots:
    Data += (Qa*M)
Qg = Data/(total_time_result)
print(f"Throughput: {Qg}")


set()
[56, 41, 46, 41, 31, 118, 48, 40, 40, 34, 53, 36, 55, 57, 59, 56, 59, 58, 59, 56, 57, 60, 57, 59, 60, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 54, 56, 58, 58, 54, 56, 57, 57, 59, 59, 58, 57, 57, 57, 58, 60, 60, 59, 59, 58, 59, 55, 60, 57, 57, 60, 57, 58, 57, 58, 59, 56, 59, 58, 55, 57, 56, 54, 56, 57, 58, 58, 56, 57, 59]
Minimal number of time slots: 118
Flow 0 starts at 0
Flow 1 starts at 0
Flow 2 starts at 0
Flow 3 starts at 0
Flow 4 starts at 0
Flow 5 starts at 0
Flow 6 starts at 0
Flow 7 starts at 0
Flow 8 starts at 0
Flow 9 starts at 0
Flow 10 starts at 0
Flow 11 starts at 0
Flow 12 starts at 0
Flow 13 starts at 0
Flow 14 starts at 0
Flow 15 starts at 0
Flow 16 starts at 0
Flow 17 starts at 0
Flow 18 starts at 0
Flow 19 starts at 0
Flow 20 starts at 0
Flow 21 starts at 0
Flow 22 starts at 0
Flow 23 starts at 0
Flow 24 starts at 0
Flow 25 starts at 0
Flow 26 starts at 0
Flow 27 starts at 0
Flow 28 starts at 0
Flow 29 starts at 0
Flow 30 starts at 0
Flow 31 starts at 0
Flow 32 

DQN UAV

In [ ]:
# Install necessary packages
!pip install gymnasium==0.28.1 stable-baselines3[extra] shimmy

# Verify the installation
import importlib
sb3_spec = importlib.util.find_spec("stable_baselines3")
if sb3_spec is None:
    print("stable-baselines3 is not installed properly.")
else:
    print("stable-baselines3 is installed correctly.")

import pandas as pd
import numpy as np
import ast
import gym
from gym import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
import torch

# Create adjacency matrix
num_flows = len(time_slots)
adjacency_matrix = np.zeros((num_flows, num_flows), dtype=int)
for i, flow_i in enumerate(flows):
    tx_i, rx_i = flow_i
    for j, flow_j in enumerate(flows):
        if i != j:
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                adjacency_matrix[i, j] = 1

# Example durations (time slots required for each flow), update as necessary
durations = time_slots

# Define the environment here to use it for processing flows and LOS data
class SchedulingEnv(gym.Env):
    def __init__(self, durations, adjacency_matrix):
        super(SchedulingEnv, self).__init__()
        self.durations = durations
        self.adjacency_matrix = adjacency_matrix
        self.num_flows = len(durations)
        self.state = None
        self.current_time_slot = 0
        self.total_reward = 0

        self.action_space = spaces.Discrete(self.num_flows)

        self.observation_space = spaces.Box(low=0, high=1, shape=(self.num_flows * 2,), dtype=np.float32)

    def reset(self):
        self.state = np.zeros(self.num_flows * 2, dtype=np.float32)
        self.current_time_slot = 0
        self.total_reward = 0
        return self.state

    def step(self, action):
        done = False
        reward = 0
        # Convert action to an integer if it's a NumPy array
        if isinstance(action, np.ndarray):
            action = action.item()  # Get the integer value from the array
        if action is not None and self.state[action] == 0:
            valid = all(self.state[j] != 1 for j in range(self.num_flows) if self.adjacency_matrix[action, j] == 1)
            if valid:
                self.state[action] = 1  # Mark as scheduled
                # Get the duration using the action index, assuming durations is a list
                self.state[self.num_flows + action] = durations[action]  # Set remaining duration
                reward += 10  # Positive reward for scheduling a flow
            else:
                reward -= 10  # Penalty for invalid action



        valid_actions = [i for i in range(self.num_flows) if self.state[i] == 0 and all(self.state[j] != 1 for j in range(self.num_flows) if self.adjacency_matrix[i, j] == 1)]

        if not valid_actions:
            for i in range(self.num_flows):
                if self.state[i] == 1 and self.state[self.num_flows + i] > 0:
                    self.state[self.num_flows + i] -= 1
                    if self.state[self.num_flows + i] == 0:
                        self.state[i] = 2  # Mark as completed
                        reward += 50  # Positive reward for completing a flow
            self.current_time_slot += 1

        if np.all(self.state[:self.num_flows] == 2):
            self.current_time_slot += max(self.state[self.num_flows:2*self.num_flows])
            done = True
            reward += 1000  # Reward for completing the scheduling

        self.total_reward += reward

        return self.state, reward, done, {}

    def render(self, mode='human'):
        pass

# Initialize the environment
env = SchedulingEnv(durations, adjacency_matrix)

# Create a vectorized environment
vec_env = make_vec_env(lambda: env, n_envs=1)

# Define and train the DQN model
model = DQN('MlpPolicy', vec_env, verbose=1)
model.learn(total_timesteps=5000)
model.save("scheduling_dqn")

# Load and test the trained model
model = DQN.load("scheduling_dqn")
obs = env.reset()

# Function to select an action such that self.state[action] = 0 and no adjacency conflicts
def select_valid_action(state, adjacency_matrix):
    valid_actions = [i for i in range(len(state)//2) if state[i] == 0 and all(state[j] != 1 for j in range(len(state)//2) if adjacency_matrix[i][j] == 1)]
    return valid_actions

for _ in range(5000):  # Reduced for quick testing
    valid_actions = select_valid_action(obs, adjacency_matrix)
    if not valid_actions:
        obs, reward, done, _ = env.step(None)
        action = None
    else:
        action, _ = model.predict(obs, deterministic=True)
        # Convert action to an integer if it's a NumPy array
        if isinstance(action, np.ndarray):
            action = action.item()  # Get the integer value from the array
        if action in valid_actions:
            obs, reward, done, _ = env.step(action)
        else:
            print(f"Selected action {action} is not valid.")
            action = None
            obs, reward, done, _ = env.step(None)
    print(f"Test - Step: {env.current_time_slot}, Action: {action}, Reward: {reward}, Total Reward: {env.total_reward}, State: {obs}")
    if done:
        obs = env.reset()

flows = flows_new

stable-baselines3 is installed correctly.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


/usr/local/lib/python3.11/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


Streaming output truncated to the last 5000 lines.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  56.   0.   0.   0.
   0. 118.  48.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.  60.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.  56.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
Selected action 21 is not valid.
Test - Step: 0, Action: None, Reward: 0, Total Reward: 50, State: [  1.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.  

In [ ]:
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    print(f"Test - Step: {env.current_time_slot}, Action: {action}, Reward: {reward}, Total Reward: {env.total_reward}, State: {obs}")

flows = flows_new

# Calculate throughput
Qa = 0.5
Data = 0
for t in time_slots:
    Data += (Qa*M)
Qg = Data/(env.current_time_slot)
print(f"Throughput: {Qg}")



Streaming output truncated to the last 5000 lines.
  0. 51.  0.  0.  0.  3.  3.  0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Test - Step: 173, Action: 74, Reward: 0, Total Reward: -79800, State: [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  1.  2.  2.  2.  2.  2.  2.  1.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  1.  2.  2.  2.  1.  2.  2.  2.  1.  1.  2.
  2.  2.  2.  1.  2.  2.  2.  2.  2.  2.  2.  0.  2.  2.  2.  0.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.
  0. 51.  0.  0.  0.  3.  3.  0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Test - Step: 173, Action: 74, Reward: 0, Total Reward: -79800, State: [ 2.  2.  2.  2.  2.

PPO UAV

In [ ]:
# Install necessary packages
!pip install gymnasium==0.28.1 stable-baselines3[extra] shimmy

# Verify the installation
import importlib
sb3_spec = importlib.util.find_spec("stable_baselines3")
if sb3_spec is None:
    print("stable-baselines3 is not installed properly.")
else:
    print("stable-baselines3 is installed correctly.")

import pandas as pd
import numpy as np
import ast
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch

# Create adjacency matrix
adjacency_matrix = np.zeros((num_flows, num_flows), dtype=int)
for i, flow_i in enumerate(flows):
    tx_i, rx_i = flow_i
    for j, flow_j in enumerate(flows):
        if i != j:
            tx_j, rx_j = flow_j
            if tx_i == tx_j or rx_i == rx_j:
                adjacency_matrix[i, j] = 1

# Calculate the throughput times for the least conflicting flows
durations = time_slots

# Initialize env variable to avoid potential issues with undefined variable
env = None

# Define the environment here to use it for processing flows and LOS data
class SchedulingEnv(gym.Env):
    def __init__(self, durations, adjacency_matrix):
        super(SchedulingEnv, self).__init__()
        self.durations = durations
        self.adjacency_matrix = adjacency_matrix
        self.num_flows = len(durations)
        self.state = None
        self.current_time_slot = 0
        self.total_reward = 0

        self.action_space = spaces.Discrete(self.num_flows)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.num_flows * 2,), dtype=np.float32)

    def reset(self):
        self.state = np.zeros(self.num_flows * 2, dtype=np.float32)
        self.current_time_slot = 0
        self.total_reward = 0
        return self.state

    def step(self, action):
        done = False
        reward = 0
        # Convert action to an integer if it's a NumPy array
        if isinstance(action, np.ndarray):
            action = action.item()  # Get the integer value from the array
        if action is not None and self.state[action] == 0:
            valid = all(self.state[j] != 1 for j in range(self.num_flows) if self.adjacency_matrix[action, j] == 1)
            if valid:
                self.state[action] = 1  # Mark as scheduled
                # Get the duration using the action index, assuming durations is a list
                self.state[self.num_flows + action] = durations[action]  # Set remaining duration
                reward += 10  # Positive reward for scheduling a flow
            else:
                reward -= 10  # Penalty for invalid action



        valid_actions = [i for i in range(self.num_flows) if self.state[i] == 0 and all(self.state[j] != 1 for j in range(self.num_flows) if self.adjacency_matrix[i, j] == 1)]

        if not valid_actions:
            for i in range(self.num_flows):
                if self.state[i] == 1 and self.state[self.num_flows + i] > 0:
                    self.state[self.num_flows + i] -= 1
                    if self.state[self.num_flows + i] == 0:
                        self.state[i] = 2  # Mark as completed
                        reward += 50  # Positive reward for completing a flow
            self.current_time_slot += 1

        if np.all(self.state[:self.num_flows] == 2):
            self.current_time_slot += max(self.state[self.num_flows:2*self.num_flows])
            done = True
            reward += 1000  # Reward for completing the scheduling

        self.total_reward += reward

        return self.state, reward, done, {}

    def render(self, mode='human'):
        pass

# Initialize the environment
env = SchedulingEnv(durations, adjacency_matrix)

# Create a vectorized environment
vec_env = make_vec_env(lambda: env, n_envs=1)

# Define and train the PPO model
model = PPO('MlpPolicy', vec_env, verbose=1)
model.learn(total_timesteps=10000)
model.save("scheduling_ppo")

# Load and test the trained model
model = PPO.load("scheduling_ppo")
obs = env.reset()

# Function to select an action such that self.state[action] = 0 and no adjacency conflicts
def select_valid_action(state, adjacency_matrix):
    valid_actions = [i for i in range(len(state)//2) if state[i] == 0 and all(state[j] != 1 for j in range(len(state)//2) if adjacency_matrix[i][j] == 1)]
    return valid_actions

for _ in range(500):  # Reduced for quick testing
    valid_actions = select_valid_action(obs, adjacency_matrix)
    if not valid_actions:
        obs, reward, done, _ = env.step(None)
        action = None
    else:
        action, _ = model.predict(obs, deterministic=True)
        if action in valid_actions:
            obs, reward, done, _ = env.step(action)
        else:
            print(f"Selected action {action} is not valid.")
            action = None
            obs, reward, done, _ = env.step(None)
    print(f"Test - Step: {env.current_time_slot}, Action: {action}, Reward: {reward}, Total Reward: {env.total_reward}, State: {obs}")
    if done:
        obs = env.reset()

flows = flows_new


stable-baselines3 is installed correctly.
Using cpu device


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
|    policy_gradient_loss | -0.0316     |
|    value_loss           | 8.03e+03    |
-----------------------------------------
Test - Step: 0, Action: 45, Reward: 10, Total Reward: 10, State: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 58.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Test - Step: 0, Action: 2, Reward: 10, Total Reward: 20, State: [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.

In [ ]:
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    print(f"Test - Step: {env.current_time_slot}, Action: {action}, Reward: {reward}, Total Reward: {env.total_reward}, State: {obs}")

flows = flows_new

# Calculate throughput
Qa = 0.5
Data = 0
for t in time_slots:
    Data += (Qa*M)
Qg = Data/(env.current_time_slot)
print(f"Throughput: {Qg}")

Streaming output truncated to the last 5000 lines.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  7.  0.
  0.  0.  0. 11.  0.  0.  0.  0. 11.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Test - Step: 166, Action: 55, Reward: 0, Total Reward: -4250, State: [ 2.  2.  2.  2.  2.  2.  2.  0.  2.  2.  0.  1.  2.  1.  2.  2.  2.  1.
  2.  1.  2.  2.  2.  2.  0.  2.  2.  2.  2.  2.  2.  0.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  0.  2.  2.  1.  2.  2.  2.  2.  1.  2.  2.  2.  2.
  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  0.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  4.  0. 45.  0.  0.  0.  1.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  7.  0.
  0.  0.  0. 11.  0.  0.  0.  0. 11.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Test - Step: 166, 